# Problema de Programación Lineal con PulP

In [1]:
# !pip install pulp

In [2]:
import numpy as np
import pulp as pl

## Problema

<img src="https://miro.medium.com/max/1050/1*pNF1FGt82jC-t7rgWDIadg.png" alt="drawing" width="400" height="500" align="center"/>

<img src="https://miro.medium.com/max/1050/1*3VQ2wm5ICw3Y8LERWSX1-Q.png" alt="drawing" width="400" height="500" align="center"/>

<img src="https://miro.medium.com/max/1050/1*R2Ka9rUBa_jmczjlFLf6Gg.png" alt="drawing" width="400" height="500" align="center"/>

<img src="https://miro.medium.com/max/561/1*sXTrgh1q6h0mpQCrpWJj4A.png" alt="drawing" width="200" height="250" align="center"/>

<img src="https://miro.medium.com/max/432/1*yi2G3GORtU6ZqFYkDoQVWw.png" alt="drawing" width="150" height="200" align="center"/>

Data source: [Towards Data Science](https://towardsdatascience.com/linear-programming-using-python-priyansh-22b5ee888fe0)

## Construcción de los vectores y matrices de datos

In [3]:
# Matriz de costos: coeficiente de la F.O.:
matriz_costos = np.array([[1, 3, 0.5, 4], 
                         [2.5, 5, 1.5, 2.5]])

# Restricciones de inventario:
restr_stock = np.array([60000,   # WH1
                        80000])  # WH2

# Restricciones de Demanda:
restr_demanda = np.array([35000,  # Customer 1
                         22000,   # Customer 2
                         18000,   # Customer 3
                         30000])  # Customer 4

In [4]:
# Construir de subindices de las variables de decisión:
dim_matriz = (restr_stock.shape[0], restr_demanda.shape[0])  # n_rows x n_cols
indices_variables = [str(i)+str(j) for j in range(1, dim_matriz[0]+1) for i in range(1, dim_matriz[1]+1)]
print("Variable Indices:", indices_variables)

Variable Indices: ['11', '21', '31', '41', '12', '22', '32', '42']


In [5]:
# Creación de matriz de variables de decisión discretas (3x4), con restricción de no negatividad
matriz_variables_int = pl.LpVariable.matrix("X", indices_variables, cat = "Integer", lowBound = 0 )
matriz_variables = np.array(matriz_variables_int).reshape(dim_matriz)
print("Variables de Decisión/Matriz de asignación:\n", matriz_variables)

Variables de Decisión/Matriz de asignación:
 [[X_11 X_21 X_31 X_41]
 [X_12 X_22 X_32 X_42]]


## Construcción del Modelo

In [6]:
# Instanciando el problema de Minimización
model_LP = pl.LpProblem(name = "Supply-Demand-Problem", sense = pl.LpMinimize) # sense options: LpMinimize or LpMaximize
model_LP

Supply-Demand-Problem:
MINIMIZE
None
VARIABLES

In [7]:
# Construir la Función Objetivo (F.O):
obj_func = pl.lpSum(matriz_variables*matriz_costos)
print(f"F.O.: min({obj_func})")

F.O.: min(X_11 + 2.5*X_12 + 3.0*X_21 + 5.0*X_22 + 0.5*X_31 + 1.5*X_32 + 4.0*X_41 + 2.5*X_42)


In [8]:
# Agregar la F.O. al modelo de PL:
model_LP +=  obj_func
print(model_LP)

Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 2.5*X_12 + 3.0*X_21 + 5.0*X_22 + 0.5*X_31 + 1.5*X_32 + 4.0*X_41 + 2.5*X_42 + 0.0
VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_31 Integer
0 <= X_32 Integer
0 <= X_41 Integer
0 <= X_42 Integer



In [9]:
# Agregar al modelo de PL las inecuaciones de restricciones de Stock:
for i in range(dim_matriz[0]):
    ineq = pl.lpSum(matriz_variables[i][j] for j in range(dim_matriz[1])) <= restr_stock[i]
    print(ineq, end='\n\n')
    model_LP += (ineq, "Restricciones de Stock " + str(i+1))

X_11 + X_21 + X_31 + X_41 <= 60000

X_12 + X_22 + X_32 + X_42 <= 80000



In [10]:
# Agregar al modelo de PL las inecuaciones de restricciones de Demanda:
for i in range(dim_matriz[1]):
    ineq = pl.lpSum(matriz_variables[j][i] for j in range(dim_matriz[0])) >= restr_demanda[i]
    print(ineq, end='\n\n')
    model_LP += (ineq, "Restricciones de Demanda " + str(i))

X_11 + X_12 >= 35000

X_21 + X_22 >= 22000

X_31 + X_32 >= 18000

X_41 + X_42 >= 30000



## Resumen del Modelo

In [11]:
model_LP

Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 2.5*X_12 + 3.0*X_21 + 5.0*X_22 + 0.5*X_31 + 1.5*X_32 + 4.0*X_41 + 2.5*X_42 + 0.0
SUBJECT TO
Restricciones_de_Stock_1: X_11 + X_21 + X_31 + X_41 <= 60000

Restricciones_de_Stock_2: X_12 + X_22 + X_32 + X_42 <= 80000

Restricciones_de_Demanda_0: X_11 + X_12 >= 35000

Restricciones_de_Demanda_1: X_21 + X_22 >= 22000

Restricciones_de_Demanda_2: X_31 + X_32 >= 18000

Restricciones_de_Demanda_3: X_41 + X_42 >= 30000

VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_31 Integer
0 <= X_32 Integer
0 <= X_41 Integer
0 <= X_42 Integer

## Resolviendo el problema

In [12]:
# Solucionador disponible con PulP
print(pl.listSolvers(onlyAvailable=True))

['PULP_CBC_CMD']


In [13]:
model_LP.solve()

status = pl.LpStatus[model_LP.status]

print(status)

Optimal


Tener en cuenta que, aunque el estado es óptimo en este caso, no tiene por qué serlo. En caso de que el problema esté mal formulado o no haya suficiente información, la solución puede ser inviable o ilimitada.

## Solución

In [14]:
print("Costo Total:", model_LP.objective.value())

# Variables de Decisión
for v in model_LP.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error, no pudo encontrar valor")

Costo Total: 200000.0
X_11 = 35000.0
X_12 = 0.0
X_21 = 22000.0
X_22 = 0.0
X_31 = 3000.0
X_32 = 15000.0
X_41 = 0.0
X_42 = 30000.0
